!python lexicais_gramaticais.py --file_path bosque25_wb.conllu --type_filter gramaticais --ncpus 7

In [2]:
!pip3 install tqdm

    100% |████████████████████████████████| 61kB 825kB/s ta 0:00:01


In [1]:
import pandas as pd
import os
import re
#from nltk.util import ngrams
import pickle
from collections import defaultdict
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool, freeze_support
import argparse
import pickle
from Restrictions import Restrictions , process_result, preprocessing

In [2]:
file_path='0-20150121-TESEMSC_0_jptdp.conllu'
type_filter='gramaticais'
ncpus=7

In [3]:
restrictions = Restrictions(type_filter)
df = preprocessing(file_path)
restrictions.calculate_restrictions(df)
sent_total = df.index[-1][0]

all_X: {'SYM', 'AUX', 'ROOT-POS', 'ADJ', 'CCONJ', 'SCONJ', 'ADV', 'NOUN', 'ADP', '_', 'X', 'PRON', 'VERB', 'DET', 'PROPN', 'NUM', 'PUNCT'}

all_PronType:{'Art', 'Ord', 'Prs', 'Range', 'Tot', 'Ind', 'Neg', 'Emp', 'Card', 'Rel', '_', 'Dem'}
setA: {'CCONJ', 'SCONJ', 'ADP', 'PRON', 'DET'}
setB: {'SYM', 'AUX', 'ROOT-POS', 'ADJ', 'ADV', 'X', 'NUM', 'VERB', 'PROPN', '_', 'NOUN'}
SetPronTypeA: {'Rel', 'Int'}
SetPronTypeB: {'Art', 'Ord', 'Prs', 'Range', 'Tot', 'Ind', 'Neg', 'Emp', 'Card', '_', 'Dem'}


In [4]:
df['l_ant_1']=df['l'].shift(periods=1)
df['l_ant_2']=df['l'].shift(periods=2)

In [5]:
df = df[~(df.l == '_')]

In [6]:
df[~(df.l=='_')].head()

w           l      x   g          f                 sent_id  \
s i                                                                         
1 1           ﻿           ﻿  PROPN  30      nsubj  0-20150121-TESEMSC_0-1   
  2  INTRODUÇÃO  INTRODUÇÃO  PROPN   1  flat:name  0-20150121-TESEMSC_0-1   
  3          Um          um    NUM   1        obj  0-20150121-TESEMSC_0-1   
  4          de          de    ADP   6       case  0-20150121-TESEMSC_0-1   
  5          os           o    DET   6        det  0-20150121-TESEMSC_0-1   

    Number type Type Gender Tense Person Voice Definite Case Mood Polarity  \
s i                                                                          
1 1   Sing    _    _      _     _      _     _        _    _    _        _   
  2   Sing    _    _      _     _      _     _        _    _    _        _   
  3   Sing    _    _      _     _      _     _        _    _    _        _   
  4      _    _    _      _     _      _     _        _    _    _        _   
  5   Plur  Art  Art   Masc     _      _     _      Def    _    _        _   

            l-x     l_ant_1 l_ant_2  
s i                                  
1 1           ﻿         NaN     NaN  
  2  INTRODUÇÃO           ﻿     NaN  
  3          um  INTRODUÇÃO       ﻿  
  4          de           _      um  
  5           o          de       _

In [7]:
import tqdm

In [8]:
with Pool(ncpus) as p:
    args = [(df, type_filter, restrictions, i) for i in range(1,sent_total+1)]
    results = list(tqdm.tqdm(p.map(process_result, args), total=len(args)))

/home/elvis/BIG-Oil-NLP/scripts/Cristian-Marneffe/Restrictions.py:109: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  pd.DataFrame([{'l-x': '<end>'}], index=['-1'])])
/home/elvis/BIG-Oil-NLP/scripts/Cristian-Marneffe/Restrictions.py:109: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  pd.DataFrame([{'l-x': '<end>'}], index=['-1'])])
/home/elvis/BIG-Oil-NLP/scripts/Cristian-Marneffe/Restrictions.py:109: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, 

In [9]:
df_results = pd.concat(results, axis=0)

In [10]:
df_results.head()

,ii,g,pair_ngram_a,pair_ngram_b,f,sent_id_int,reverse,type_a,type_b,x_a,x_b,x,sent_id
1,1,30,,,nsubj,1,False,,,,,PROPN,0-20150121-TESEMSC_0-1
2,2,1,,,flat:name,1,False,,,,,PROPN,0-20150121-TESEMSC_0-1
3,3,1,,,obj,1,False,,,,,NUM,0-20150121-TESEMSC_0-1
4,4,6,"(um, de)","(fator, que)",case,1,False,_,_,ADP,NOUN,ADP,0-20150121-TESEMSC_0-1
5,5,6,"(de, o)","(fator, que)",det,1,False,Art,_,DET,NOUN,DET,0-20150121-TESEMSC_0-1


In [11]:
#pair_ngram_freq = df_results.groupby(['ii','g','pair_ngram_a','pair_ngram_b','f','sent_id_int','reverse','type_a','type_b',
#                   'x_a','x_b','x','sent_id'])
df_results = df_results[~((df_results.pair_ngram_a=='') & (df_results.pair_ngram_b==''))]
df_results['condition'] = df_results.apply(lambda row:(row.pair_ngram_a,row.pair_ngram_b,row.reverse),axis=1)
pair_ngram_freq = df_results.groupby('condition')
#pair_ngram_freq={}
#for group_name, df_group in groups:
#    pair_ngram_freq[group_name] = df_group[['reverse','sent_id_int','ii','g', 'type_a','type_b', 'sent_id', 'x_a', 'x_b']]

In [12]:
relevant_pair_ngram_freq ={group_name:df_group for group_name , df_group in  pair_ngram_freq if len(df_group)>1}

In [13]:
len(relevant_pair_ngram_freq)

924

pair_ngram_freq=defaultdict(list)
total_results_lines = len(df_results.index)
for ind in df_results.index:
    group_ngram_distance = df_results.ix[ind]
    i, j, ngram_a, ngram_b, relation , sent_id, reverse, type_a, type_b, x_a, x_b, x, sent_id_str = group_ngram_distance
    if ngram_a=='' and ngram_b=='':
        continue
    pair_ngram_freq[(ngram_a, ngram_b, reverse)].append((relation,sent_id,i,j, type_a, type_b, sent_id_str, x_a, x_b))

In [14]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [15]:
def get_colored_word(row, word1, word2):
    #if '-' in row.name :
    #    continue
    if row.name == word1:
        bold = '<font color="red"><b>{}</b></font>'.format(row.w)
        return bold #sentence.append(bold)
    elif row.name == word2:
        bold = '<font color="blue"><b>{}</b></font>'.format(row.w)
        return bold #sentence.append(bold)
    else:
        return row.w #sentence.append()

In [16]:
from collections import Counter

examples = []
for ind, (group_name, df_group) in enumerate(relevant_pair_ngram_freq.items()):
    if ind%100==0:
        print(ind, end=' ')
    count_relations = Counter(list(df_group.f.values))
    # Caso exista várias relações
    if len(count_relations)>1:
        sentences=[]
        for ind,case_row in df_group.iterrows():
            sent_id = case_row.sent_id_int
            word1 = case_row.ii
            word2 = case_row.g

            sentence = list(df.loc[sent_id].apply(lambda row: get_colored_word(row, word1, word2), axis=1).values)       
            #sentences.append((sentence , rel, sent_id, type_a, type_b, sent_id_str, x_a, x_b))
            sentences.append((sentence , case_row))#rel, sent_id, type_a, type_b, sent_id_str, x_a, x_b))
        examples.append((sentences, group_name, count_relations))

0 100 200 300 400 500 600 700 800 900 

In [17]:
if not os.path.isdir("results"):
    os.mkdir("results")

#reverse if nn[2] else 
if type_filter=='gramaticais':
    html_file = 'results/results_gramaticais.html'
    reverse = "<span style='font-size:25px;'>&#8678;</span>"
    no_reverse = "<span style='font-size:25px;'>&#8680;</span>"
    with open(html_file,'w+',encoding="utf-8") as file:

        for i,more_than_sentences in enumerate(examples):
            sentences,nn,a = more_than_sentences
            blue_text = '<font color="blue"><b>{}</b></font>'
            red_text = '<font color="red"><b>{}</b></font>'
            file.write('<p><b>Exemplo {}:</b> ( {} , {} ) <b>{}</b> ( {} , {} )</p>'.format(i+1, 
                                                                    nn[0][0], 
                                                                    blue_text.format(nn[0][1]) if nn[2] else  red_text.format(nn[0][1]), 
                                                                    no_reverse,\
                                                                    red_text.format(nn[1][0]) if nn[2] else  blue_text.format(nn[1][0]), 
                                                                    nn[1][1]))
            counter  = " , ".join(['<b>{}</b> : {}'.format(k,v) for k,v in a.items()])
            file.write('<p>[ {} ]</p>'.format(counter))
            for more_than_sent in sentences:
                #    'ii','g','pair_ngram_a','pair_ngram_b','f','sent_id_int','reverse','type_a','type_b',
                #    'x_a','x_b','x','sent_id'
                sent, case_row = more_than_sent
                #sent , rel, sent_id, type_a, type_b, sent_id_str, x_a, x_b = more_than_sent
                sent_id_str  = case_row.sent_id
                rel = case_row.f
                type_a = case_row.type_a
                type_b = case_row.type_b
                x_a = case_row.x_a
                x_b = case_row.x_b
                sentence = "<p><b>Frase {}</b> \t: {} - rel: <b>({})</b> ,  type: <b>({} - {})</b> , pos: <b>({} - {})</b> </p>".format(
                    sent_id_str, " ".join(sent), rel,
                    blue_text.format(type_a) if nn[2] else  red_text.format(type_a),
                    red_text.format(type_b) if nn[2] else  blue_text.format(type_b),
                    blue_text.format(x_a) if nn[2] else  red_text.format(x_a),
                    red_text.format(x_b) if nn[2] else  blue_text.format(x_b))

                file.write('{}\n'.format(sentence))
            file.write('<br>')
        
elif type_filter=='lexicais':
    html_file = 'results/results_lexicais.html'
    reverse = "<span style='font-size:25px;'>&#8678;</span>"
    no_reverse = "<span style='font-size:25px;'>&#8680;</span>"
    with open(html_file,'w+',encoding="utf-8") as file:
        i=0
        for more_than_sentences in examples:

            pair = more_than_sentences[1][:2]
            #if pair in all_pairs:
            #    continue 
            i+=1    
            sentences,nn,a = more_than_sentences
            blue_text = '<font color="blue"><b>{}</b></font>'
            red_text = '<font color="red"><b>{}</b></font>'
            file.write('<p><b>Exemplo {}:</b> {} <b>{}</b>  {}  </p>'.format(i, 
                                                                    blue_text.format(nn[0]) if nn[2] else  red_text.format(nn[0]), 
                                                                    no_reverse,\
                                                                    red_text.format(nn[1]) if nn[2] else  blue_text.format(nn[1])))
            counter  = " , ".join(['<b>{}</b> : {}'.format(k,v) for k,v in a.items()])
            file.write('<p>[ {} ]</p>'.format(counter))
            for more_than_sent in sentences:
                #    'ii','g','pair_ngram_a','pair_ngram_b','f','sent_id_int','reverse','type_a','type_b',
                #    'x_a','x_b','x','sent_id'
                sent, case_row = more_than_sent
                #sent , rel, sent_id, type_a, type_b, sent_id_str, x_a, x_b = more_than_sent
                sent_id_str  = case_row.sent_id
                rel = case_row.f
                type_a = case_row.type_a
                type_b = case_row.type_b
                x_a = case_row.x_a
                x_b = case_row.x_b
                sentence = "<p><b>Frase {}</b> \t: {} - rel: <b>({})</b> ,  type: <b>({} - {})</b> , pos: <b>({} - {})</b> </p>".format(
                    sent_id_str, " ".join(sent), rel,
                    blue_text.format(type_a) if nn[2] else  red_text.format(type_a),
                    red_text.format(type_b) if nn[2] else  blue_text.format(type_b),
                    blue_text.format(x_a) if nn[2] else  red_text.format(x_a),
                    red_text.format(x_b) if nn[2] else  blue_text.format(x_b))
                file.write('{}\n'.format(sentence))
            file.write('<br>')
else:
    raise Exception('unknown filter type (gramaticais, lexicais): {}'.format(type_filter))

In [19]:
sent

['Em',
 'o',
 'Brasil',
 ',',
 'isso',
 'vem',
 'sendo',
 'feito',
 'em',
 'especial',
 'em',
 'o',
 'que',
 '<font color="red"><b>se</b></font>',
 '<font color="blue"><b>refere</b></font>',
 'a',
 'os',
 'combustíveis',
 'automotivos',
 ',',
 'em',
 'paralelo',
 'a',
 'o',
 'programa',
 'de',
 'acompanhamento',
 'de',
 'a',
 'qualidade',
 'de',
 'os',
 'produtos',
 'comercializados',
 '(',
 'CIÊNCIA',
 'HOJE',
 ',',
 '2000',
 ')',
 '.']